# Detecation
## Prepare requirements

In [ ]:
# clone yolov7 repo
!git clone https://github.com/WongKinYiu/yolov7.git

In [ ]:
# fix requirements.txt
fileContent = ""
with open("./yolov7/requirements.txt", "r") as file:
    for line in file:
        if line.startswith("numpy"):
            continue
        fileContent += line

with open ("./yolov7/requirements.txt", "w") as file:
    file.write(fileContent)

In [ ]:
# install dependencies
!pip install -r ./yolov7/requirements.txt

In [ ]:
# install newest numpy version manually
!pip install numpy

In [ ]:
# install dependency to collect images
!pip install simple-image-download

## Acquire dataset

In [ ]:
# import downloader
from simple_image_download import simple_image_download as sid

response = sid.Downloader()

In [ ]:
# define which queries to look for online
downloadQueries = ["cat", "playing-cats", "2-cats"]

In [ ]:
# download images
"""for query in downloadQueries:
    response.download(query, limit=100)"""

response.download(downloadQueries[0], limit=100)

In [ ]:
response.download(downloadQueries[1], limit=100)

In [ ]:
response.download(downloadQueries[2], limit=70)

In [ ]:
from os import listdir, remove, rename, mkdir

# remove unwanted images in the wrong format
for query in downloadQueries:
    for image in listdir(f"./simple_images/{query}"):
        if not image.endswith(".jpeg"):
            remove(f"./simple_images/{query}/{image}")

# create new directory for all images
mkdir("./images")

# move all images to one folder
for query in downloadQueries:
    for image in listdir(f"./simple_images/{query}"):
        rename(f"./simple_images/{query}/{image}", f"./images/{image}")